In [1]:
import torch
import random
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import pandas as pd

from data_loader_darts import get_dataloaders_simple
# from darts_search_bdp import train_darts_search_bdp
from darts_search_bdp import train_darts_search_bdp
from model_build import FinalNetwork
from cell_plot import plot_cell

def set_seed(seed=42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True

Error.  nthreads cannot be larger than environment variable "NUMEXPR_MAX_THREADS" (64)

In [2]:

# 1. Set random seed
set_seed(42)
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 2. Load data
print("[INFO] Loading 50/50 split data...")
train_loader, val_loader, num_classes = get_dataloaders_simple(batch_size=32)
print(f"[INFO] DARTS will run on {len(train_loader.dataset.y)} train samples and {len(val_loader.dataset.y)} val samples")

[INFO] Loading 50/50 split data...
[DEBUG] Loaded ./PSG/SC4001E0.npz → 841 samples
[DEBUG] Loaded ./PSG/SC4002E0.npz → 1127 samples
[DEBUG] Loaded ./PSG/SC4011E0.npz → 1103 samples
[DEBUG] Loaded ./PSG/SC4012E0.npz → 1186 samples
[DEBUG] Loaded ./PSG/SC4021E0.npz → 1025 samples
[DEBUG] Loaded ./PSG/SC4022E0.npz → 1009 samples
[DEBUG] Loaded ./PSG/SC4031E0.npz → 952 samples
[DEBUG] Loaded ./PSG/SC4032E0.npz → 911 samples
[DEBUG] Loaded ./PSG/SC4041E0.npz → 1235 samples
[DEBUG] Loaded ./PSG/SC4042E0.npz → 1200 samples
[DEBUG] Loaded ./PSG/SC4051E0.npz → 672 samples
[DEBUG] Loaded ./PSG/SC4052E0.npz → 1246 samples
[DEBUG] Loaded ./PSG/SC4061E0.npz → 843 samples
[DEBUG] Loaded ./PSG/SC4062E0.npz → 1016 samples
[DEBUG] Loaded ./PSG/SC4071E0.npz → 976 samples
[DEBUG] Loaded ./PSG/SC4072E0.npz → 1273 samples
[DEBUG] Loaded ./PSG/SC4081E0.npz → 1134 samples
[DEBUG] Loaded ./PSG/SC4082E0.npz → 1054 samples
[DEBUG] Loaded ./PSG/SC4091E0.npz → 1132 samples
[DEBUG] Loaded ./PSG/SC4092E0.npz → 1105

In [ ]:
# 3. Run DARTS search with pruning
print("[INFO] Running DARTS search with BDP...")
searched_genotype, pruned_train_loader, pruned_val_loader = train_darts_search_bdp(
    train_loader, val_loader, num_classes,
    epochs=25, prune_every=5, pt=0.05, pv=0.05,
    device=device
)

In [2]:
from collections import namedtuple

Genotype = namedtuple('Genotype', 'normal normal_concat reduce reduce_concat')

searched_genotype = Genotype(
    normal=[
        ('dil_conv_1x5', 1),
        ('dil_conv_1x5', 0),
        ('sep_conv_1x5', 0),
        ('sep_conv_1x5', 1),
        ('max_pool_3x3', 2),
        ('max_pool_3x3', 1),
        ('max_pool_3x3', 4),
        ('max_pool_3x3', 2),
        ('dil_conv_1x5', 4),
        ('dil_conv_1x5', 3)
    ],
    normal_concat=[0, 1, 2, 3, 4],
    
    reduce=[
        ('sep_conv_1x3', 1),
        ('sep_conv_1x5', 0),
        ('dil_conv_1x5', 0),
        ('max_pool_3x3', 2),
        ('max_pool_3x3', 3),
        ('max_pool_3x3', 0),
        ('max_pool_3x3', 4),
        ('max_pool_3x3', 3),
        ('max_pool_3x3', 5),
        ('max_pool_3x3', 4)
    ],
    reduce_concat=[0, 1, 2, 3, 4]
)


In [31]:
!sudo apt update
!sudo apt install -y graphviz graphviz-dev
!pip install pygraphviz

# 4. Visualize searched cells
print("[INFO] Visualizing searched cells...")
plot_cell(searched_genotype, 'normal')
plot_cell(searched_genotype, 'reduce')

Get:1 https://deb.nodesource.com/node_20.x nodistro InRelease [12.1 kB]
Get:2 https://deb.nodesource.com/node_20.x nodistro/main amd64 Packages [12.4 kB]0m
Get:3 http://archive.ubuntu.com/ubuntu jammy InRelease [270 kB]                
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]      
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]m3m
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy/main amd64 Packages [1792 kB]
Get:8 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [3207 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy/universe amd64 Packages [17.5 MB]m
Get:10 http://archive.ubuntu.com/ubuntu jammy/restricted amd64 Packages [164 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy/multiverse amd64 Packages [266 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1575 kB]m
Get:13 http://archive.ubuntu.com/ubuntu jamm

/home/work/Duc/NASEEG2/NASEEG/SleepC/sleep_nas/cell_plot.py:26: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()


In [ ]:
import torch
import pandas as pd
import numpy as np

print("[INFO] Running 5-Fold Cross Validation on pruned data...")

# === Hàm trích xuất toàn bộ dữ liệu từ DataLoader ===
def extract_from_loader(loader):
    X_list, y_list = [], []
    for x, y in loader:
        X_list.append(x.cpu())
        y_list.append(y.cpu())
    return torch.cat(X_list, dim=0), torch.cat(y_list, dim=0)

# === Trích xuất X, y từ train và val loader ===
X_train_all, y_train_all = extract_from_loader(pruned_train_loader)
X_val_all,   y_val_all   = extract_from_loader(pruned_val_loader)

# Gộp train + val
X_all = torch.cat([X_train_all, X_val_all], dim=0)  # shape: (N, C, T) hoặc (N, T)
y_all = torch.cat([y_train_all, y_val_all], dim=0)  # shape: (N,)

# === Chuyển về numpy ===
X_np = X_all.numpy()
y_np = y_all.numpy().reshape(-1, 1)

# === Reshape X về (N, D) nếu cần (flatten nếu có chiều phụ) ===
if X_np.ndim > 2:
    X_np = X_np.reshape(X_np.shape[0], -1)  # (N, D)

# ❌ KHÔNG chuẩn hóa, giữ nguyên raw X_np

# === Ghép lại X và y ===
data_np = np.hstack((X_np, y_np))  # shape: (N, D+1)

# === Tạo DataFrame và lưu CSV ===
num_features = X_np.shape[1]
column_names = [f"feature_{i}" for i in range(num_features)] + ["label"]
df = pd.DataFrame(data_np, columns=column_names)

df.to_csv("pruned_dataset.csv", index=False)
print("✅ Đã lưu dữ liệu gốc (KHÔNG chuẩn hóa) vào 'pruned_dataset.csv'")


[INFO] Running 5-Fold Cross Validation on pruned data...
✅ Đã lưu dữ liệu gốc (KHÔNG chuẩn hóa) vào 'pruned_dataset.csv'


In [2]:
import pandas as pd
import numpy as np
df = pd.read_csv("pruned_dataset_78.csv")
X_np = df.drop(columns=["label"]).values
y_np = df["label"].values
unique, counts = np.unique(y_np, return_counts=True)

print("\n[INFO] Number of samples per class:")
for label, count in zip(unique, counts):
    print(f"  Class {label}: {count} samples")



[INFO] Number of samples per class:
  Class 0: 53407 samples
  Class 1: 20446 samples
  Class 2: 57828 samples
  Class 3: 12388 samples
  Class 4: 23252 samples


In [ ]:
# Post-search filter pruning based on information capacity and independence (Stage 2)
# === Entropy-based pruning ===
def compute_filter_entropy(weight_tensor):
    entropy_list = []
    for filt in weight_tensor:
        filt_flat = filt.view(filt.size(0), -1)
        norms = torch.norm(filt_flat, dim=1) + 1e-6
        p = norms / norms.sum()
        entropy = -torch.sum(p * torch.log2(p))
        entropy_list.append(entropy.item())
    return entropy_list

def prune_model_entropy(model, prune_ratio=0.5):
    for name, module in model.named_modules():
        if isinstance(module, nn.Conv1d):
            weight = module.weight.data.detach().cpu()
            entropy = compute_filter_entropy(weight)
            entropy_tensor = torch.tensor(entropy)
            k = int((1 - prune_ratio) * len(entropy))
            topk_indices = torch.topk(entropy_tensor, k=k).indices
            mask = torch.zeros_like(entropy_tensor)
            mask[topk_indices] = 1.0
            full_mask = mask[:, None, None].expand_as(weight).to(module.weight.device)
            module.weight.data *= full_mask
    return model

def count_pruned_weights(model):
    total, nonzero = 0, 0
    for module in model.modules():
        if isinstance(module, (nn.Conv1d, nn.Linear)):
            w = module.weight.data
            total += w.numel()
            nonzero += w.nonzero().size(0)
    zero = total - nonzero
    print(f"[INFO] Total weights: {total}")
    print(f"[INFO] Non-zero weights: {nonzero}")
    print(f"[INFO] Pruned weights: {zero}")
    print(f"[INFO] Pruned ratio: {100 * zero / total:.2f}%")

def evaluate_metrics(y_true, y_pred, num_classes):
    acc = accuracy_score(y_true, y_pred)
    mf1 = f1_score(y_true, y_pred, average='macro', zero_division=0)
    prec = precision_score(y_true, y_pred, average=None, zero_division=0)
    rec = recall_score(y_true, y_pred, average=None, zero_division=0)
    f1s = f1_score(y_true, y_pred, average=None, zero_division=0)
    gmeans = []

    for c in range(num_classes):
        tp = np.sum((y_pred == c) & (y_true == c))
        fn = np.sum((y_pred != c) & (y_true == c))
        recall_c = tp / (tp + fn) if (tp + fn) > 0 else 0
        gmeans.append(recall_c)

    mgm = np.sqrt(np.prod(gmeans)) if np.all(np.array(gmeans) > 0) else 0.0
    cm = confusion_matrix(y_true, y_pred)
    return acc, mf1, mgm, prec, rec, f1s, gmeans, cm

In [23]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
from torch.utils.data import TensorDataset, DataLoader
import matplotlib.pyplot as plt
from model_build import FinalNetwork

# ==== Load and Normalize Data ====
df = pd.read_csv("pruned_dataset.csv")
X_np = df.drop(columns=["label"]).values
y_np = df["label"].values

# Z-score normalization
X_mean = X_np.mean(axis=0)
X_std = np.where(X_np.std(axis=0) == 0, 1, X_np.std(axis=0))
X_z = (X_np - X_mean) / X_std
X_z = np.clip(X_z, -3, 3) / 3.0

X_all = torch.tensor(X_z, dtype=torch.float32).unsqueeze(1)
y_all = torch.tensor(y_np, dtype=torch.long)

num_classes = len(np.unique(y_np))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ==== Evaluation Metric Function ====
def evaluate_metrics(y_true, y_pred, num_classes):
    acc = accuracy_score(y_true, y_pred)
    mf1 = f1_score(y_true, y_pred, average='macro', zero_division=0)
    prec = precision_score(y_true, y_pred, average=None, zero_division=0)
    rec = recall_score(y_true, y_pred, average=None, zero_division=0)
    f1s = f1_score(y_true, y_pred, average=None, zero_division=0)

    gmeans = []
    for c in range(num_classes):
        tp = np.sum((y_pred == c) & (y_true == c))
        fn = np.sum((y_pred != c) & (y_true == c))
        tn = np.sum((y_pred != c) & (y_true != c))
        fp = np.sum((y_pred == c) & (y_true != c))
        gm = np.sqrt((tp / (tp + fn + 1e-6)) * (tn / (tn + fp + 1e-6)))
        gmeans.append(gm)
    cm = confusion_matrix(y_true, y_pred)
    return acc, mf1, np.mean(gmeans), prec, rec, f1s, gmeans, cm

# ==== K-Fold Training ====
kfold = KFold(n_splits=5, shuffle=True, random_state=47)
all_val_true, all_val_pred = [], []

for fold_idx, (train_idx, val_idx) in enumerate(kfold.split(X_all)):
    print(f"\n===== Fold {fold_idx + 1} =====")

    X_train, y_train = X_all[train_idx], y_all[train_idx]
    X_val, y_val = X_all[val_idx], y_all[val_idx]

    train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=64, shuffle=True)
    val_loader = DataLoader(TensorDataset(X_val, y_val), batch_size=64, shuffle=False)

    model = FinalNetwork(C=9, num_classes=num_classes, layers=7, genotype=searched_genotype).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
    criterion = nn.CrossEntropyLoss()

    best_val_acc = 0.0
    best_result = {}
    no_improve_counter = 0  # ← EARLY STOP counter

    for epoch in range(50):
        model.train()
        train_loss = 0
        train_true, train_pred = [], []

        for x, y in train_loader:
            x, y = x.to(device).squeeze(-1), y.to(device)
            optimizer.zero_grad()
            output = model(x)
            loss = criterion(output, y)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            train_true.extend(y.cpu().numpy())
            train_pred.extend(output.argmax(dim=1).cpu().numpy())

        model.eval()
        val_loss = 0
        val_true, val_pred = [], []
        with torch.no_grad():
            for x, y in val_loader:
                x, y = x.to(device).squeeze(-1), y.to(device)
                output = model(x)
                loss = criterion(output, y)
                val_loss += loss.item()
                val_true.extend(y.cpu().numpy())
                val_pred.extend(output.argmax(dim=1).cpu().numpy())

        train_acc = accuracy_score(train_true, train_pred)
        val_acc = accuracy_score(val_true, val_pred)

        print(f"[Fold {fold_idx+1} | Epoch {epoch+1}] "
              f"Train Loss: {train_loss/len(train_loader):.4f} | Train Acc: {train_acc:.4f} | "
              f"Val Loss: {val_loss/len(val_loader):.4f} | Val Acc: {val_acc:.4f}")

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            no_improve_counter = 0  # reset counter
            acc, mf1, mgm, prec, rec, f1s, gmeans, cm = evaluate_metrics(np.array(val_true), np.array(val_pred), num_classes)
            best_result = {
                'epoch': epoch + 1,
                'acc': acc,
                'mf1': mf1,
                'gmean': mgm,
                'prec': prec,
                'rec': rec,
                'f1': f1s,
                'gmean_class': gmeans,
                'cm': cm,
                'val_true': val_true,
                'val_pred': val_pred
            }
        else:
            no_improve_counter += 1
            if no_improve_counter >= 10:
                print(f"[Early Stopping] No improvement in 10 epochs. Stopping at epoch {epoch+1}.")
                break

    # === Print Best Result for Fold ===
    print(f"\n===== BEST RESULT FOR FOLD {fold_idx+1} =====")
    print(f"Best Epoch: {best_result['epoch']}")
    print(f"ACC: {best_result['acc']:.4f} | MF1: {best_result['mf1']:.4f} | G-Mean: {best_result['gmean']:.4f}")
    for i in range(num_classes):
        print(f"[Class {i}] Prec: {best_result['prec'][i]:.4f} | Rec: {best_result['rec'][i]:.4f} "
              f"| F1: {best_result['f1'][i]:.4f} | GM: {best_result['gmean_class'][i]:.4f}")

    all_val_true.extend(best_result['val_true'])
    all_val_pred.extend(best_result['val_pred'])

# ==== FINAL EVALUATION ON MERGED VAL SET ====
acc, mf1, mgm, prec, rec, f1s, gmeans, cm = evaluate_metrics(np.array(all_val_true), np.array(all_val_pred), num_classes)

print("\n===== FINAL EVALUATION ON MERGED TEST SET (AFTER K-FOLD) =====")
print(f"ACC: {acc:.4f} | MF1: {mf1:.4f} | G-Mean: {mgm:.4f}")
for i in range(num_classes):
    print(f"[Class {i}] Prec: {prec[i]:.4f} | Rec: {rec[i]:.4f} | F1: {f1s[i]:.4f} | GM: {gmeans[i]:.4f}")
print("Confusion Matrix:")
print(cm)



===== Fold 1 =====
[Fold 1 | Epoch 1] Train Loss: 0.5881 | Train Acc: 0.7674 | Val Loss: 0.5197 | Val Acc: 0.8065
[Fold 1 | Epoch 2] Train Loss: 0.5273 | Train Acc: 0.7935 | Val Loss: 0.5245 | Val Acc: 0.8017
[Fold 1 | Epoch 3] Train Loss: 0.5118 | Train Acc: 0.8011 | Val Loss: 0.4812 | Val Acc: 0.8120
[Fold 1 | Epoch 4] Train Loss: 0.4980 | Train Acc: 0.8062 | Val Loss: 0.4819 | Val Acc: 0.8143
[Fold 1 | Epoch 5] Train Loss: 0.4891 | Train Acc: 0.8086 | Val Loss: 0.5630 | Val Acc: 0.7838
[Fold 1 | Epoch 6] Train Loss: 0.4821 | Train Acc: 0.8121 | Val Loss: 0.4602 | Val Acc: 0.8210
[Fold 1 | Epoch 7] Train Loss: 0.4744 | Train Acc: 0.8164 | Val Loss: 0.4610 | Val Acc: 0.8225
[Fold 1 | Epoch 8] Train Loss: 0.4709 | Train Acc: 0.8176 | Val Loss: 0.4630 | Val Acc: 0.8224
[Fold 1 | Epoch 9] Train Loss: 0.4656 | Train Acc: 0.8190 | Val Loss: 0.4462 | Val Acc: 0.8290
[Fold 1 | Epoch 10] Train Loss: 0.4606 | Train Acc: 0.8209 | Val Loss: 0.4578 | Val Acc: 0.8240
[Fold 1 | Epoch 11] Train Los

[Fold 3 | Epoch 6] Train Loss: 0.4811 | Train Acc: 0.8127 | Val Loss: 0.4752 | Val Acc: 0.8170
[Fold 3 | Epoch 7] Train Loss: 0.4758 | Train Acc: 0.8156 | Val Loss: 0.4588 | Val Acc: 0.8237
[Fold 3 | Epoch 8] Train Loss: 0.4710 | Train Acc: 0.8168 | Val Loss: 0.4709 | Val Acc: 0.8179
[Fold 3 | Epoch 9] Train Loss: 0.4649 | Train Acc: 0.8200 | Val Loss: 0.4469 | Val Acc: 0.8283
[Fold 3 | Epoch 10] Train Loss: 0.4594 | Train Acc: 0.8208 | Val Loss: 0.4725 | Val Acc: 0.8182
[Fold 3 | Epoch 11] Train Loss: 0.4552 | Train Acc: 0.8234 | Val Loss: 0.4389 | Val Acc: 0.8285
[Fold 3 | Epoch 12] Train Loss: 0.4508 | Train Acc: 0.8248 | Val Loss: 0.4535 | Val Acc: 0.8262
[Fold 3 | Epoch 13] Train Loss: 0.4476 | Train Acc: 0.8274 | Val Loss: 0.4847 | Val Acc: 0.8085
[Fold 3 | Epoch 14] Train Loss: 0.4439 | Train Acc: 0.8273 | Val Loss: 0.5504 | Val Acc: 0.7908
[Fold 3 | Epoch 15] Train Loss: 0.4387 | Train Acc: 0.8296 | Val Loss: 0.4408 | Val Acc: 0.8298
[Fold 3 | Epoch 16] Train Loss: 0.4371 | Tra

[Fold 5 | Epoch 26] Train Loss: 0.3996 | Train Acc: 0.8451 | Val Loss: 0.4890 | Val Acc: 0.8121
[Fold 5 | Epoch 27] Train Loss: 0.3947 | Train Acc: 0.8461 | Val Loss: 0.4607 | Val Acc: 0.8224
[Fold 5 | Epoch 28] Train Loss: 0.3922 | Train Acc: 0.8484 | Val Loss: 0.4549 | Val Acc: 0.8275
[Fold 5 | Epoch 29] Train Loss: 0.3892 | Train Acc: 0.8498 | Val Loss: 0.4619 | Val Acc: 0.8260
[Fold 5 | Epoch 30] Train Loss: 0.3845 | Train Acc: 0.8505 | Val Loss: 0.4853 | Val Acc: 0.8142
[Fold 5 | Epoch 31] Train Loss: 0.3811 | Train Acc: 0.8527 | Val Loss: 0.5063 | Val Acc: 0.8147
[Early Stopping] No improvement in 10 epochs. Stopping at epoch 31.

===== BEST RESULT FOR FOLD 5 =====
Best Epoch: 21
ACC: 0.8285 | MF1: 0.7754 | G-Mean: 0.8532
[Class 0] Prec: 0.9360 | Rec: 0.9254 | F1: 0.9307 | GM: 0.9477
[Class 1] Prec: 0.5951 | Rec: 0.3882 | F1: 0.4699 | GM: 0.6114
[Class 2] Prec: 0.8543 | Rec: 0.8845 | F1: 0.8691 | GM: 0.9022
[Class 3] Prec: 0.8690 | Rec: 0.8426 | F1: 0.8556 | GM: 0.9131
[Class 4] 

In [26]:
#9_1
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
from torch.utils.data import TensorDataset, DataLoader
import matplotlib.pyplot as plt
from model_build import FinalNetwork

# ==== Load and Normalize Data ====
df = pd.read_csv("pruned_dataset.csv")
X_np = df.drop(columns=["label"]).values
y_np = df["label"].values

# Z-score normalization
X_mean = X_np.mean(axis=0)
X_std = np.where(X_np.std(axis=0) == 0, 1, X_np.std(axis=0))
X_z = (X_np - X_mean) / X_std
X_z = np.clip(X_z, -3, 3) / 3.0

X_all = torch.tensor(X_z, dtype=torch.float32).unsqueeze(1)
y_all = torch.tensor(y_np, dtype=torch.long)

num_classes = len(np.unique(y_np))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ==== Evaluation Metric Function ====
def evaluate_metrics(y_true, y_pred, num_classes):
    acc = accuracy_score(y_true, y_pred)
    mf1 = f1_score(y_true, y_pred, average='macro', zero_division=0)
    prec = precision_score(y_true, y_pred, average=None, zero_division=0)
    rec = recall_score(y_true, y_pred, average=None, zero_division=0)
    f1s = f1_score(y_true, y_pred, average=None, zero_division=0)

    gmeans = []
    for c in range(num_classes):
        tp = np.sum((y_pred == c) & (y_true == c))
        fn = np.sum((y_pred != c) & (y_true == c))
        tn = np.sum((y_pred != c) & (y_true != c))
        fp = np.sum((y_pred == c) & (y_true != c))
        gm = np.sqrt((tp / (tp + fn + 1e-6)) * (tn / (tn + fp + 1e-6)))
        gmeans.append(gm)
    cm = confusion_matrix(y_true, y_pred)
    return acc, mf1, np.mean(gmeans), prec, rec, f1s, gmeans, cm

# ==== K-Fold Training ====
kfold = KFold(n_splits=5, shuffle=True, random_state=47)
all_val_true, all_val_pred = [], []

for fold_idx, (train_idx, val_idx) in enumerate(kfold.split(X_all)):
    print(f"\n===== Fold {fold_idx + 1} =====")

    X_train, y_train = X_all[train_idx], y_all[train_idx]
    X_val, y_val = X_all[val_idx], y_all[val_idx]

    train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=64, shuffle=True)
    val_loader = DataLoader(TensorDataset(X_val, y_val), batch_size=64, shuffle=False)

    model = FinalNetwork(C=9, num_classes=num_classes, layers=9, genotype=searched_genotype).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
    criterion = nn.CrossEntropyLoss()

    best_val_acc = 0.0
    best_result = {}
    no_improve_counter = 0  # ← EARLY STOP counter

    for epoch in range(50):
        model.train()
        train_loss = 0
        train_true, train_pred = [], []

        for x, y in train_loader:
            x, y = x.to(device).squeeze(-1), y.to(device)
            optimizer.zero_grad()
            output = model(x)
            loss = criterion(output, y)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            train_true.extend(y.cpu().numpy())
            train_pred.extend(output.argmax(dim=1).cpu().numpy())

        model.eval()
        val_loss = 0
        val_true, val_pred = [], []
        with torch.no_grad():
            for x, y in val_loader:
                x, y = x.to(device).squeeze(-1), y.to(device)
                output = model(x)
                loss = criterion(output, y)
                val_loss += loss.item()
                val_true.extend(y.cpu().numpy())
                val_pred.extend(output.argmax(dim=1).cpu().numpy())

        train_acc = accuracy_score(train_true, train_pred)
        val_acc = accuracy_score(val_true, val_pred)

        print(f"[Fold {fold_idx+1} | Epoch {epoch+1}] "
              f"Train Loss: {train_loss/len(train_loader):.4f} | Train Acc: {train_acc:.4f} | "
              f"Val Loss: {val_loss/len(val_loader):.4f} | Val Acc: {val_acc:.4f}")

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            no_improve_counter = 0  # reset counter
            acc, mf1, mgm, prec, rec, f1s, gmeans, cm = evaluate_metrics(np.array(val_true), np.array(val_pred), num_classes)
            best_result = {
                'epoch': epoch + 1,
                'acc': acc,
                'mf1': mf1,
                'gmean': mgm,
                'prec': prec,
                'rec': rec,
                'f1': f1s,
                'gmean_class': gmeans,
                'cm': cm,
                'val_true': val_true,
                'val_pred': val_pred
            }
        else:
            no_improve_counter += 1
            if no_improve_counter >= 10:
                print(f"[Early Stopping] No improvement in 10 epochs. Stopping at epoch {epoch+1}.")
                break

    # === Print Best Result for Fold ===
    print(f"\n===== BEST RESULT FOR FOLD {fold_idx+1} =====")
    print(f"Best Epoch: {best_result['epoch']}")
    print(f"ACC: {best_result['acc']:.4f} | MF1: {best_result['mf1']:.4f} | G-Mean: {best_result['gmean']:.4f}")
    for i in range(num_classes):
        print(f"[Class {i}] Prec: {best_result['prec'][i]:.4f} | Rec: {best_result['rec'][i]:.4f} "
              f"| F1: {best_result['f1'][i]:.4f} | GM: {best_result['gmean_class'][i]:.4f}")

    all_val_true.extend(best_result['val_true'])
    all_val_pred.extend(best_result['val_pred'])

# ==== FINAL EVALUATION ON MERGED VAL SET ====
acc, mf1, mgm, prec, rec, f1s, gmeans, cm = evaluate_metrics(np.array(all_val_true), np.array(all_val_pred), num_classes)

print("\n===== FINAL EVALUATION ON MERGED TEST SET (AFTER K-FOLD) =====")
print(f"ACC: {acc:.4f} | MF1: {mf1:.4f} | G-Mean: {mgm:.4f}")
for i in range(num_classes):
    print(f"[Class {i}] Prec: {prec[i]:.4f} | Rec: {rec[i]:.4f} | F1: {f1s[i]:.4f} | GM: {gmeans[i]:.4f}")
print("Confusion Matrix:")
print(cm)



===== Fold 1 =====
[Fold 1 | Epoch 1] Train Loss: 0.5977 | Train Acc: 0.7635 | Val Loss: 0.5472 | Val Acc: 0.7883
[Fold 1 | Epoch 2] Train Loss: 0.5327 | Train Acc: 0.7911 | Val Loss: 0.4989 | Val Acc: 0.8062
[Fold 1 | Epoch 3] Train Loss: 0.5154 | Train Acc: 0.7997 | Val Loss: 0.5013 | Val Acc: 0.8058
[Fold 1 | Epoch 4] Train Loss: 0.5025 | Train Acc: 0.8038 | Val Loss: 0.4796 | Val Acc: 0.8162
[Fold 1 | Epoch 5] Train Loss: 0.4952 | Train Acc: 0.8069 | Val Loss: 0.4587 | Val Acc: 0.8242
[Fold 1 | Epoch 6] Train Loss: 0.4872 | Train Acc: 0.8101 | Val Loss: 0.5139 | Val Acc: 0.8022
[Fold 1 | Epoch 7] Train Loss: 0.4823 | Train Acc: 0.8126 | Val Loss: 0.4519 | Val Acc: 0.8275
[Fold 1 | Epoch 8] Train Loss: 0.4762 | Train Acc: 0.8153 | Val Loss: 0.4690 | Val Acc: 0.8180
[Fold 1 | Epoch 9] Train Loss: 0.4722 | Train Acc: 0.8155 | Val Loss: 0.4520 | Val Acc: 0.8256
[Fold 1 | Epoch 10] Train Loss: 0.4667 | Train Acc: 0.8189 | Val Loss: 0.4679 | Val Acc: 0.8175
[Fold 1 | Epoch 11] Train Los

[Fold 3 | Epoch 13] Train Loss: 0.4573 | Train Acc: 0.8223 | Val Loss: 0.4610 | Val Acc: 0.8211
[Fold 3 | Epoch 14] Train Loss: 0.4532 | Train Acc: 0.8230 | Val Loss: 0.4577 | Val Acc: 0.8227
[Fold 3 | Epoch 15] Train Loss: 0.4489 | Train Acc: 0.8263 | Val Loss: 0.4448 | Val Acc: 0.8287
[Fold 3 | Epoch 16] Train Loss: 0.4444 | Train Acc: 0.8282 | Val Loss: 0.4511 | Val Acc: 0.8252
[Fold 3 | Epoch 17] Train Loss: 0.4417 | Train Acc: 0.8289 | Val Loss: 0.4411 | Val Acc: 0.8299
[Fold 3 | Epoch 18] Train Loss: 0.4373 | Train Acc: 0.8297 | Val Loss: 0.4503 | Val Acc: 0.8232
[Fold 3 | Epoch 19] Train Loss: 0.4353 | Train Acc: 0.8310 | Val Loss: 0.4423 | Val Acc: 0.8312
[Fold 3 | Epoch 20] Train Loss: 0.4308 | Train Acc: 0.8328 | Val Loss: 0.4462 | Val Acc: 0.8283
[Fold 3 | Epoch 21] Train Loss: 0.4275 | Train Acc: 0.8344 | Val Loss: 0.4836 | Val Acc: 0.8074
[Fold 3 | Epoch 22] Train Loss: 0.4265 | Train Acc: 0.8350 | Val Loss: 0.4892 | Val Acc: 0.8153
[Fold 3 | Epoch 23] Train Loss: 0.4229 |

[Fold 5 | Epoch 19] Train Loss: 0.4307 | Train Acc: 0.8329 | Val Loss: 0.4765 | Val Acc: 0.8187
[Early Stopping] No improvement in 10 epochs. Stopping at epoch 19.

===== BEST RESULT FOR FOLD 5 =====
Best Epoch: 9
ACC: 0.8240 | MF1: 0.7806 | G-Mean: 0.8558
[Class 0] Prec: 0.9452 | Rec: 0.9102 | F1: 0.9273 | GM: 0.9422
[Class 1] Prec: 0.5258 | Rec: 0.5145 | F1: 0.5201 | GM: 0.6934
[Class 2] Prec: 0.8463 | Rec: 0.8845 | F1: 0.8650 | GM: 0.8997
[Class 3] Prec: 0.8999 | Rec: 0.7941 | F1: 0.8437 | GM: 0.8879
[Class 4] Prec: 0.7270 | Rec: 0.7679 | F1: 0.7469 | GM: 0.8559

===== FINAL EVALUATION ON MERGED TEST SET (AFTER K-FOLD) =====
ACC: 0.8302 | MF1: 0.7852 | G-Mean: 0.8588
[Class 0] Prec: 0.9236 | Rec: 0.9313 | F1: 0.9274 | GM: 0.9474
[Class 1] Prec: 0.5490 | Rec: 0.4914 | F1: 0.5186 | GM: 0.6810
[Class 2] Prec: 0.8575 | Rec: 0.8813 | F1: 0.8692 | GM: 0.9017
[Class 3] Prec: 0.8845 | Rec: 0.8249 | F1: 0.8536 | GM: 0.9043
[Class 4] Prec: 0.7425 | Rec: 0.7719 | F1: 0.7569 | GM: 0.8594
Confus

In [27]:
#p9_2
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
from torch.utils.data import TensorDataset, DataLoader

# ==== Load and Normalize Data ====
df = pd.read_csv("pruned_dataset.csv")
X_np = df.drop(columns=["label"]).values
y_np = df["label"].values

X_mean = X_np.mean(axis=0)
X_std = np.where(X_np.std(axis=0) == 0, 1, X_np.std(axis=0))
X_z = (X_np - X_mean) / X_std
X_z = np.clip(X_z, -3, 3) / 3.0

X_all = torch.tensor(X_z, dtype=torch.float32).unsqueeze(1)
y_all = torch.tensor(y_np, dtype=torch.long)

num_classes = len(np.unique(y_np))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ==== K-Fold Training ====
kfold = KFold(n_splits=5, shuffle=True, random_state=42)
all_val_true, all_val_pred = [], []

for fold_idx, (train_idx, val_idx) in enumerate(kfold.split(X_all)):
    print(f"\n===== Fold {fold_idx + 1} =====")

    # === Build and Prune Model ===
    model_unpruned = FinalNetwork(C=9, num_classes=num_classes, layers=9, genotype=searched_genotype).to(device)
    total_params_before = sum(p.numel() for p in model_unpruned.parameters())
    print(f"[INFO] Total parameters BEFORE pruning: {total_params_before:,}")

    model = prune_model_entropy(model_unpruned, prune_ratio=0.5)
    nonzero_params_after = sum((p != 0).sum().item() for p in model.parameters())
    zero_params = total_params_before - nonzero_params_after
    pruned_ratio = 100 * zero_params / total_params_before
    print(f"[INFO] Non-zero parameters AFTER pruning: {nonzero_params_after:,}")
    print(f"[INFO] Pruned parameters: {zero_params:,} ({pruned_ratio:.2f}%)")

    optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
    criterion = nn.CrossEntropyLoss()

    # Dataloader
    X_train, y_train = X_all[train_idx], y_all[train_idx]
    X_val, y_val = X_all[val_idx], y_all[val_idx]
    train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=64, shuffle=True)
    val_loader = DataLoader(TensorDataset(X_val, y_val), batch_size=64, shuffle=False)

    best_val_acc = 0.0
    best_result = {}
    no_improve_counter = 0  # ← EARLY STOPPING COUNTER

    for epoch in range(50):  # Max epochs
        model.train()
        train_loss = 0
        train_true, train_pred = [], []

        for x, y in train_loader:
            x, y = x.to(device).squeeze(-1), y.to(device)
            optimizer.zero_grad()
            output = model(x)
            loss = criterion(output, y)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            train_true.extend(y.cpu().numpy())
            train_pred.extend(output.argmax(dim=1).cpu().numpy())

        model.eval()
        val_loss = 0
        val_true, val_pred = [], []
        with torch.no_grad():
            for x, y in val_loader:
                x, y = x.to(device).squeeze(-1), y.to(device)
                output = model(x)
                loss = criterion(output, y)
                val_loss += loss.item()
                val_true.extend(y.cpu().numpy())
                val_pred.extend(output.argmax(dim=1).cpu().numpy())

        train_acc = accuracy_score(train_true, train_pred)
        val_acc = accuracy_score(val_true, val_pred)

        print(f"[Fold {fold_idx+1} | Epoch {epoch+1}] Train Loss: {train_loss/len(train_loader):.4f} | "
              f"Train Acc: {train_acc:.4f} | Val Loss: {val_loss/len(val_loader):.4f} | Val Acc: {val_acc:.4f}")

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            no_improve_counter = 0  # reset counter
            acc, mf1, mgm, prec, rec, f1s, gmeans, cm = evaluate_metrics(
                np.array(val_true), np.array(val_pred), num_classes
            )
            best_result = {
                'epoch': epoch + 1,
                'acc': acc,
                'mf1': mf1,
                'gmean': mgm,
                'prec': prec,
                'rec': rec,
                'f1': f1s,
                'gmean_class': gmeans,
                'cm': cm,
                'val_true': val_true,
                'val_pred': val_pred
            }
        else:
            no_improve_counter += 1
            if no_improve_counter >= 10:
                print(f"[Early Stopping] No improvement in 10 consecutive epochs. Stopping at epoch {epoch+1}.")
                break

    # === Print Best Result for Fold ===
    print(f"\n===== BEST RESULT FOR FOLD {fold_idx+1} =====")
    print(f"Best Epoch: {best_result['epoch']}")
    print(f"ACC: {best_result['acc']:.4f} | MF1: {best_result['mf1']:.4f} | G-Mean: {best_result['gmean']:.4f}")
    for i in range(num_classes):
        print(f"[Class {i}] Prec: {best_result['prec'][i]:.4f} | Rec: {best_result['rec'][i]:.4f} "
              f"| F1: {best_result['f1'][i]:.4f} | GM: {best_result['gmean_class'][i]:.4f}")

    # Gộp toàn bộ val_true và val_pred để đánh giá toàn bộ tập sau K-Fold
    all_val_true.extend(best_result['val_true'])
    all_val_pred.extend(best_result['val_pred'])

# ==== FINAL EVALUATION ON MERGED VAL SET ====
acc, mf1, mgm, prec, rec, f1s, gmeans, cm = evaluate_metrics(np.array(all_val_true), np.array(all_val_pred), num_classes)

print("\n===== FINAL EVALUATION ON MERGED TEST SET (AFTER K-FOLD) =====")
print(f"ACC: {acc:.4f} | MF1: {mf1:.4f} | G-Mean: {mgm:.4f}")
for i in range(num_classes):
    print(f"[Class {i}] Prec: {prec[i]:.4f} | Rec: {rec[i]:.4f} | F1: {f1s[i]:.4f} | GM: {gmeans[i]:.4f}")
print("Confusion Matrix:")
print(cm)



===== Fold 1 =====
[INFO] Total parameters BEFORE pruning: 84,812
[INFO] Non-zero parameters AFTER pruning: 42,608
[INFO] Pruned parameters: 42,204 (49.76%)
[Fold 1 | Epoch 1] Train Loss: 0.6056 | Train Acc: 0.7606 | Val Loss: 0.6019 | Val Acc: 0.7646
[Fold 1 | Epoch 2] Train Loss: 0.5375 | Train Acc: 0.7893 | Val Loss: 0.5465 | Val Acc: 0.7903
[Fold 1 | Epoch 3] Train Loss: 0.5211 | Train Acc: 0.7964 | Val Loss: 0.5361 | Val Acc: 0.7921
[Fold 1 | Epoch 4] Train Loss: 0.5086 | Train Acc: 0.8015 | Val Loss: 0.4933 | Val Acc: 0.8106
[Fold 1 | Epoch 5] Train Loss: 0.5000 | Train Acc: 0.8047 | Val Loss: 0.4856 | Val Acc: 0.8128
[Fold 1 | Epoch 6] Train Loss: 0.4940 | Train Acc: 0.8077 | Val Loss: 0.5566 | Val Acc: 0.7845
[Fold 1 | Epoch 7] Train Loss: 0.4888 | Train Acc: 0.8105 | Val Loss: 0.5159 | Val Acc: 0.7986
[Fold 1 | Epoch 8] Train Loss: 0.4829 | Train Acc: 0.8120 | Val Loss: 0.4700 | Val Acc: 0.8184
[Fold 1 | Epoch 9] Train Loss: 0.4801 | Train Acc: 0.8130 | Val Loss: 0.4630 | Val

[Fold 3 | Epoch 5] Train Loss: 0.5075 | Train Acc: 0.8020 | Val Loss: 0.5398 | Val Acc: 0.7815
[Fold 3 | Epoch 6] Train Loss: 0.5011 | Train Acc: 0.8040 | Val Loss: 0.4940 | Val Acc: 0.8084
[Fold 3 | Epoch 7] Train Loss: 0.4924 | Train Acc: 0.8095 | Val Loss: 0.4782 | Val Acc: 0.8142
[Fold 3 | Epoch 8] Train Loss: 0.4876 | Train Acc: 0.8108 | Val Loss: 0.4792 | Val Acc: 0.8145
[Fold 3 | Epoch 9] Train Loss: 0.4828 | Train Acc: 0.8116 | Val Loss: 0.4848 | Val Acc: 0.8099
[Fold 3 | Epoch 10] Train Loss: 0.4793 | Train Acc: 0.8135 | Val Loss: 0.4873 | Val Acc: 0.8114
[Fold 3 | Epoch 11] Train Loss: 0.4723 | Train Acc: 0.8170 | Val Loss: 0.4723 | Val Acc: 0.8145
[Fold 3 | Epoch 12] Train Loss: 0.4693 | Train Acc: 0.8175 | Val Loss: 0.4531 | Val Acc: 0.8237
[Fold 3 | Epoch 13] Train Loss: 0.4667 | Train Acc: 0.8176 | Val Loss: 0.4518 | Val Acc: 0.8252
[Fold 3 | Epoch 14] Train Loss: 0.4635 | Train Acc: 0.8208 | Val Loss: 0.4635 | Val Acc: 0.8189
[Fold 3 | Epoch 15] Train Loss: 0.4603 | Trai

[Fold 5 | Epoch 11] Train Loss: 0.4717 | Train Acc: 0.8173 | Val Loss: 0.4544 | Val Acc: 0.8223
[Fold 5 | Epoch 12] Train Loss: 0.4678 | Train Acc: 0.8183 | Val Loss: 0.4626 | Val Acc: 0.8196
[Fold 5 | Epoch 13] Train Loss: 0.4642 | Train Acc: 0.8204 | Val Loss: 0.4628 | Val Acc: 0.8218
[Fold 5 | Epoch 14] Train Loss: 0.4604 | Train Acc: 0.8218 | Val Loss: 0.4841 | Val Acc: 0.8151
[Fold 5 | Epoch 15] Train Loss: 0.4584 | Train Acc: 0.8227 | Val Loss: 0.4484 | Val Acc: 0.8273
[Fold 5 | Epoch 16] Train Loss: 0.4539 | Train Acc: 0.8236 | Val Loss: 0.4733 | Val Acc: 0.8124
[Fold 5 | Epoch 17] Train Loss: 0.4512 | Train Acc: 0.8246 | Val Loss: 0.4714 | Val Acc: 0.8128
[Fold 5 | Epoch 18] Train Loss: 0.4477 | Train Acc: 0.8267 | Val Loss: 0.4744 | Val Acc: 0.8125
[Fold 5 | Epoch 19] Train Loss: 0.4465 | Train Acc: 0.8276 | Val Loss: 0.4618 | Val Acc: 0.8211
[Fold 5 | Epoch 20] Train Loss: 0.4424 | Train Acc: 0.8283 | Val Loss: 0.4533 | Val Acc: 0.8251
[Fold 5 | Epoch 21] Train Loss: 0.4409 |

In [28]:
#11_1
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
from torch.utils.data import TensorDataset, DataLoader
import matplotlib.pyplot as plt
from model_build import FinalNetwork

# ==== Load and Normalize Data ====
df = pd.read_csv("pruned_dataset.csv")
X_np = df.drop(columns=["label"]).values
y_np = df["label"].values

# Z-score normalization
X_mean = X_np.mean(axis=0)
X_std = np.where(X_np.std(axis=0) == 0, 1, X_np.std(axis=0))
X_z = (X_np - X_mean) / X_std
X_z = np.clip(X_z, -3, 3) / 3.0

X_all = torch.tensor(X_z, dtype=torch.float32).unsqueeze(1)
y_all = torch.tensor(y_np, dtype=torch.long)

num_classes = len(np.unique(y_np))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ==== Evaluation Metric Function ====
def evaluate_metrics(y_true, y_pred, num_classes):
    acc = accuracy_score(y_true, y_pred)
    mf1 = f1_score(y_true, y_pred, average='macro', zero_division=0)
    prec = precision_score(y_true, y_pred, average=None, zero_division=0)
    rec = recall_score(y_true, y_pred, average=None, zero_division=0)
    f1s = f1_score(y_true, y_pred, average=None, zero_division=0)

    gmeans = []
    for c in range(num_classes):
        tp = np.sum((y_pred == c) & (y_true == c))
        fn = np.sum((y_pred != c) & (y_true == c))
        tn = np.sum((y_pred != c) & (y_true != c))
        fp = np.sum((y_pred == c) & (y_true != c))
        gm = np.sqrt((tp / (tp + fn + 1e-6)) * (tn / (tn + fp + 1e-6)))
        gmeans.append(gm)
    cm = confusion_matrix(y_true, y_pred)
    return acc, mf1, np.mean(gmeans), prec, rec, f1s, gmeans, cm

# ==== K-Fold Training ====
kfold = KFold(n_splits=5, shuffle=True, random_state=47)
all_val_true, all_val_pred = [], []

for fold_idx, (train_idx, val_idx) in enumerate(kfold.split(X_all)):
    print(f"\n===== Fold {fold_idx + 1} =====")

    X_train, y_train = X_all[train_idx], y_all[train_idx]
    X_val, y_val = X_all[val_idx], y_all[val_idx]

    train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=64, shuffle=True)
    val_loader = DataLoader(TensorDataset(X_val, y_val), batch_size=64, shuffle=False)

    model = FinalNetwork(C=9, num_classes=num_classes, layers=11, genotype=searched_genotype).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
    criterion = nn.CrossEntropyLoss()

    best_val_acc = 0.0
    best_result = {}
    no_improve_counter = 0  # ← EARLY STOP counter

    for epoch in range(50):
        model.train()
        train_loss = 0
        train_true, train_pred = [], []

        for x, y in train_loader:
            x, y = x.to(device).squeeze(-1), y.to(device)
            optimizer.zero_grad()
            output = model(x)
            loss = criterion(output, y)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            train_true.extend(y.cpu().numpy())
            train_pred.extend(output.argmax(dim=1).cpu().numpy())

        model.eval()
        val_loss = 0
        val_true, val_pred = [], []
        with torch.no_grad():
            for x, y in val_loader:
                x, y = x.to(device).squeeze(-1), y.to(device)
                output = model(x)
                loss = criterion(output, y)
                val_loss += loss.item()
                val_true.extend(y.cpu().numpy())
                val_pred.extend(output.argmax(dim=1).cpu().numpy())

        train_acc = accuracy_score(train_true, train_pred)
        val_acc = accuracy_score(val_true, val_pred)

        print(f"[Fold {fold_idx+1} | Epoch {epoch+1}] "
              f"Train Loss: {train_loss/len(train_loader):.4f} | Train Acc: {train_acc:.4f} | "
              f"Val Loss: {val_loss/len(val_loader):.4f} | Val Acc: {val_acc:.4f}")

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            no_improve_counter = 0  # reset counter
            acc, mf1, mgm, prec, rec, f1s, gmeans, cm = evaluate_metrics(np.array(val_true), np.array(val_pred), num_classes)
            best_result = {
                'epoch': epoch + 1,
                'acc': acc,
                'mf1': mf1,
                'gmean': mgm,
                'prec': prec,
                'rec': rec,
                'f1': f1s,
                'gmean_class': gmeans,
                'cm': cm,
                'val_true': val_true,
                'val_pred': val_pred
            }
        else:
            no_improve_counter += 1
            if no_improve_counter >= 10:
                print(f"[Early Stopping] No improvement in 10 epochs. Stopping at epoch {epoch+1}.")
                break

    # === Print Best Result for Fold ===
    print(f"\n===== BEST RESULT FOR FOLD {fold_idx+1} =====")
    print(f"Best Epoch: {best_result['epoch']}")
    print(f"ACC: {best_result['acc']:.4f} | MF1: {best_result['mf1']:.4f} | G-Mean: {best_result['gmean']:.4f}")
    for i in range(num_classes):
        print(f"[Class {i}] Prec: {best_result['prec'][i]:.4f} | Rec: {best_result['rec'][i]:.4f} "
              f"| F1: {best_result['f1'][i]:.4f} | GM: {best_result['gmean_class'][i]:.4f}")

    all_val_true.extend(best_result['val_true'])
    all_val_pred.extend(best_result['val_pred'])

# ==== FINAL EVALUATION ON MERGED VAL SET ====
acc, mf1, mgm, prec, rec, f1s, gmeans, cm = evaluate_metrics(np.array(all_val_true), np.array(all_val_pred), num_classes)

print("\n===== FINAL EVALUATION ON MERGED TEST SET (AFTER K-FOLD) =====")
print(f"ACC: {acc:.4f} | MF1: {mf1:.4f} | G-Mean: {mgm:.4f}")
for i in range(num_classes):
    print(f"[Class {i}] Prec: {prec[i]:.4f} | Rec: {rec[i]:.4f} | F1: {f1s[i]:.4f} | GM: {gmeans[i]:.4f}")
print("Confusion Matrix:")
print(cm)



===== Fold 1 =====
[Fold 1 | Epoch 1] Train Loss: 0.5998 | Train Acc: 0.7633 | Val Loss: 0.5385 | Val Acc: 0.7907
[Fold 1 | Epoch 2] Train Loss: 0.5339 | Train Acc: 0.7907 | Val Loss: 0.5080 | Val Acc: 0.8035
[Fold 1 | Epoch 3] Train Loss: 0.5132 | Train Acc: 0.8004 | Val Loss: 0.4999 | Val Acc: 0.8055
[Fold 1 | Epoch 4] Train Loss: 0.5005 | Train Acc: 0.8058 | Val Loss: 0.4916 | Val Acc: 0.8084
[Fold 1 | Epoch 5] Train Loss: 0.4942 | Train Acc: 0.8069 | Val Loss: 0.4859 | Val Acc: 0.8122
[Fold 1 | Epoch 6] Train Loss: 0.4855 | Train Acc: 0.8105 | Val Loss: 0.4795 | Val Acc: 0.8137
[Fold 1 | Epoch 7] Train Loss: 0.4807 | Train Acc: 0.8128 | Val Loss: 0.4551 | Val Acc: 0.8248
[Fold 1 | Epoch 8] Train Loss: 0.4739 | Train Acc: 0.8159 | Val Loss: 0.4588 | Val Acc: 0.8234
[Fold 1 | Epoch 9] Train Loss: 0.4680 | Train Acc: 0.8183 | Val Loss: 0.4664 | Val Acc: 0.8188
[Fold 1 | Epoch 10] Train Loss: 0.4635 | Train Acc: 0.8190 | Val Loss: 0.4700 | Val Acc: 0.8179
[Fold 1 | Epoch 11] Train Los

[Fold 3 | Epoch 11] Train Loss: 0.4641 | Train Acc: 0.8205 | Val Loss: 0.4538 | Val Acc: 0.8250
[Fold 3 | Epoch 12] Train Loss: 0.4580 | Train Acc: 0.8223 | Val Loss: 0.4577 | Val Acc: 0.8222
[Fold 3 | Epoch 13] Train Loss: 0.4533 | Train Acc: 0.8237 | Val Loss: 0.4757 | Val Acc: 0.8148
[Fold 3 | Epoch 14] Train Loss: 0.4514 | Train Acc: 0.8258 | Val Loss: 0.4663 | Val Acc: 0.8221
[Fold 3 | Epoch 15] Train Loss: 0.4460 | Train Acc: 0.8260 | Val Loss: 0.4442 | Val Acc: 0.8273
[Fold 3 | Epoch 16] Train Loss: 0.4431 | Train Acc: 0.8281 | Val Loss: 0.4534 | Val Acc: 0.8294
[Fold 3 | Epoch 17] Train Loss: 0.4398 | Train Acc: 0.8300 | Val Loss: 0.4538 | Val Acc: 0.8238
[Fold 3 | Epoch 18] Train Loss: 0.4368 | Train Acc: 0.8304 | Val Loss: 0.4570 | Val Acc: 0.8241
[Fold 3 | Epoch 19] Train Loss: 0.4330 | Train Acc: 0.8327 | Val Loss: 0.4442 | Val Acc: 0.8286
[Fold 3 | Epoch 20] Train Loss: 0.4298 | Train Acc: 0.8332 | Val Loss: 0.4398 | Val Acc: 0.8309
[Fold 3 | Epoch 21] Train Loss: 0.4264 |

In [29]:
#p11_2
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
from torch.utils.data import TensorDataset, DataLoader

# ==== Load and Normalize Data ====
df = pd.read_csv("pruned_dataset.csv")
X_np = df.drop(columns=["label"]).values
y_np = df["label"].values

X_mean = X_np.mean(axis=0)
X_std = np.where(X_np.std(axis=0) == 0, 1, X_np.std(axis=0))
X_z = (X_np - X_mean) / X_std
X_z = np.clip(X_z, -3, 3) / 3.0

X_all = torch.tensor(X_z, dtype=torch.float32).unsqueeze(1)
y_all = torch.tensor(y_np, dtype=torch.long)

num_classes = len(np.unique(y_np))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ==== K-Fold Training ====
kfold = KFold(n_splits=5, shuffle=True, random_state=42)
all_val_true, all_val_pred = [], []

for fold_idx, (train_idx, val_idx) in enumerate(kfold.split(X_all)):
    print(f"\n===== Fold {fold_idx + 1} =====")

    # === Build and Prune Model ===
    model_unpruned = FinalNetwork(C=9, num_classes=num_classes, layers=11, genotype=searched_genotype).to(device)
    total_params_before = sum(p.numel() for p in model_unpruned.parameters())
    print(f"[INFO] Total parameters BEFORE pruning: {total_params_before:,}")

    model = prune_model_entropy(model_unpruned, prune_ratio=0.5)
    nonzero_params_after = sum((p != 0).sum().item() for p in model.parameters())
    zero_params = total_params_before - nonzero_params_after
    pruned_ratio = 100 * zero_params / total_params_before
    print(f"[INFO] Non-zero parameters AFTER pruning: {nonzero_params_after:,}")
    print(f"[INFO] Pruned parameters: {zero_params:,} ({pruned_ratio:.2f}%)")

    optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
    criterion = nn.CrossEntropyLoss()

    # Dataloader
    X_train, y_train = X_all[train_idx], y_all[train_idx]
    X_val, y_val = X_all[val_idx], y_all[val_idx]
    train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=64, shuffle=True)
    val_loader = DataLoader(TensorDataset(X_val, y_val), batch_size=64, shuffle=False)

    best_val_acc = 0.0
    best_result = {}
    no_improve_counter = 0  # ← EARLY STOPPING COUNTER

    for epoch in range(50):  # Max epochs
        model.train()
        train_loss = 0
        train_true, train_pred = [], []

        for x, y in train_loader:
            x, y = x.to(device).squeeze(-1), y.to(device)
            optimizer.zero_grad()
            output = model(x)
            loss = criterion(output, y)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            train_true.extend(y.cpu().numpy())
            train_pred.extend(output.argmax(dim=1).cpu().numpy())

        model.eval()
        val_loss = 0
        val_true, val_pred = [], []
        with torch.no_grad():
            for x, y in val_loader:
                x, y = x.to(device).squeeze(-1), y.to(device)
                output = model(x)
                loss = criterion(output, y)
                val_loss += loss.item()
                val_true.extend(y.cpu().numpy())
                val_pred.extend(output.argmax(dim=1).cpu().numpy())

        train_acc = accuracy_score(train_true, train_pred)
        val_acc = accuracy_score(val_true, val_pred)

        print(f"[Fold {fold_idx+1} | Epoch {epoch+1}] Train Loss: {train_loss/len(train_loader):.4f} | "
              f"Train Acc: {train_acc:.4f} | Val Loss: {val_loss/len(val_loader):.4f} | Val Acc: {val_acc:.4f}")

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            no_improve_counter = 0  # reset counter
            acc, mf1, mgm, prec, rec, f1s, gmeans, cm = evaluate_metrics(
                np.array(val_true), np.array(val_pred), num_classes
            )
            best_result = {
                'epoch': epoch + 1,
                'acc': acc,
                'mf1': mf1,
                'gmean': mgm,
                'prec': prec,
                'rec': rec,
                'f1': f1s,
                'gmean_class': gmeans,
                'cm': cm,
                'val_true': val_true,
                'val_pred': val_pred
            }
        else:
            no_improve_counter += 1
            if no_improve_counter >= 10:
                print(f"[Early Stopping] No improvement in 10 consecutive epochs. Stopping at epoch {epoch+1}.")
                break

    # === Print Best Result for Fold ===
    print(f"\n===== BEST RESULT FOR FOLD {fold_idx+1} =====")
    print(f"Best Epoch: {best_result['epoch']}")
    print(f"ACC: {best_result['acc']:.4f} | MF1: {best_result['mf1']:.4f} | G-Mean: {best_result['gmean']:.4f}")
    for i in range(num_classes):
        print(f"[Class {i}] Prec: {best_result['prec'][i]:.4f} | Rec: {best_result['rec'][i]:.4f} "
              f"| F1: {best_result['f1'][i]:.4f} | GM: {best_result['gmean_class'][i]:.4f}")

    # Gộp toàn bộ val_true và val_pred để đánh giá toàn bộ tập sau K-Fold
    all_val_true.extend(best_result['val_true'])
    all_val_pred.extend(best_result['val_pred'])

# ==== FINAL EVALUATION ON MERGED VAL SET ====
acc, mf1, mgm, prec, rec, f1s, gmeans, cm = evaluate_metrics(np.array(all_val_true), np.array(all_val_pred), num_classes)

print("\n===== FINAL EVALUATION ON MERGED TEST SET (AFTER K-FOLD) =====")
print(f"ACC: {acc:.4f} | MF1: {mf1:.4f} | G-Mean: {mgm:.4f}")
for i in range(num_classes):
    print(f"[Class {i}] Prec: {prec[i]:.4f} | Rec: {rec[i]:.4f} | F1: {f1s[i]:.4f} | GM: {gmeans[i]:.4f}")
print("Confusion Matrix:")
print(cm)



===== Fold 1 =====
[INFO] Total parameters BEFORE pruning: 117,212
[INFO] Non-zero parameters AFTER pruning: 58,808
[INFO] Pruned parameters: 58,404 (49.83%)
[Fold 1 | Epoch 1] Train Loss: 0.6085 | Train Acc: 0.7594 | Val Loss: 0.5731 | Val Acc: 0.7762
[Fold 1 | Epoch 2] Train Loss: 0.5432 | Train Acc: 0.7874 | Val Loss: 0.5784 | Val Acc: 0.7668
[Fold 1 | Epoch 3] Train Loss: 0.5252 | Train Acc: 0.7950 | Val Loss: 0.5975 | Val Acc: 0.7693
[Fold 1 | Epoch 4] Train Loss: 0.5143 | Train Acc: 0.8005 | Val Loss: 0.5166 | Val Acc: 0.7965
[Fold 1 | Epoch 5] Train Loss: 0.5050 | Train Acc: 0.8036 | Val Loss: 0.4844 | Val Acc: 0.8147
[Fold 1 | Epoch 6] Train Loss: 0.4968 | Train Acc: 0.8058 | Val Loss: 0.4950 | Val Acc: 0.8128
[Fold 1 | Epoch 7] Train Loss: 0.4907 | Train Acc: 0.8080 | Val Loss: 0.5055 | Val Acc: 0.7989
[Fold 1 | Epoch 8] Train Loss: 0.4861 | Train Acc: 0.8114 | Val Loss: 0.4712 | Val Acc: 0.8169
[Fold 1 | Epoch 9] Train Loss: 0.4798 | Train Acc: 0.8140 | Val Loss: 0.4649 | Va

[INFO] Non-zero parameters AFTER pruning: 58,808
[INFO] Pruned parameters: 58,404 (49.83%)
[Fold 3 | Epoch 1] Train Loss: 0.6090 | Train Acc: 0.7580 | Val Loss: 0.5551 | Val Acc: 0.7799
[Fold 3 | Epoch 2] Train Loss: 0.5434 | Train Acc: 0.7873 | Val Loss: 0.5481 | Val Acc: 0.7872
[Fold 3 | Epoch 3] Train Loss: 0.5230 | Train Acc: 0.7961 | Val Loss: 0.5418 | Val Acc: 0.7820
[Fold 3 | Epoch 4] Train Loss: 0.5119 | Train Acc: 0.8001 | Val Loss: 0.4794 | Val Acc: 0.8137
[Fold 3 | Epoch 5] Train Loss: 0.5034 | Train Acc: 0.8033 | Val Loss: 0.4800 | Val Acc: 0.8139
[Fold 3 | Epoch 6] Train Loss: 0.4962 | Train Acc: 0.8062 | Val Loss: 0.4862 | Val Acc: 0.8111
[Fold 3 | Epoch 7] Train Loss: 0.4915 | Train Acc: 0.8085 | Val Loss: 0.4706 | Val Acc: 0.8177
[Fold 3 | Epoch 8] Train Loss: 0.4861 | Train Acc: 0.8111 | Val Loss: 0.5034 | Val Acc: 0.8047
[Fold 3 | Epoch 9] Train Loss: 0.4822 | Train Acc: 0.8129 | Val Loss: 0.4885 | Val Acc: 0.8105
[Fold 3 | Epoch 10] Train Loss: 0.4755 | Train Acc: 0.

[INFO] Non-zero parameters AFTER pruning: 58,808
[INFO] Pruned parameters: 58,404 (49.83%)
[Fold 5 | Epoch 1] Train Loss: 0.6227 | Train Acc: 0.7542 | Val Loss: 0.5237 | Val Acc: 0.7950
[Fold 5 | Epoch 2] Train Loss: 0.5463 | Train Acc: 0.7858 | Val Loss: 0.6075 | Val Acc: 0.7661
[Fold 5 | Epoch 3] Train Loss: 0.5296 | Train Acc: 0.7939 | Val Loss: 0.5225 | Val Acc: 0.7946
[Fold 5 | Epoch 4] Train Loss: 0.5179 | Train Acc: 0.7981 | Val Loss: 0.4960 | Val Acc: 0.8093
[Fold 5 | Epoch 5] Train Loss: 0.5116 | Train Acc: 0.8010 | Val Loss: 0.4941 | Val Acc: 0.8110
[Fold 5 | Epoch 6] Train Loss: 0.5023 | Train Acc: 0.8038 | Val Loss: 0.5004 | Val Acc: 0.8047
[Fold 5 | Epoch 7] Train Loss: 0.4961 | Train Acc: 0.8064 | Val Loss: 0.5267 | Val Acc: 0.7960
[Fold 5 | Epoch 8] Train Loss: 0.4901 | Train Acc: 0.8081 | Val Loss: 0.4728 | Val Acc: 0.8157
[Fold 5 | Epoch 9] Train Loss: 0.4849 | Train Acc: 0.8109 | Val Loss: 0.4712 | Val Acc: 0.8188
[Fold 5 | Epoch 10] Train Loss: 0.4802 | Train Acc: 0.

In [30]:
#p11_2
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
from torch.utils.data import TensorDataset, DataLoader

# ==== Load and Normalize Data ====
df = pd.read_csv("pruned_dataset.csv")
X_np = df.drop(columns=["label"]).values
y_np = df["label"].values

X_mean = X_np.mean(axis=0)
X_std = np.where(X_np.std(axis=0) == 0, 1, X_np.std(axis=0))
X_z = (X_np - X_mean) / X_std
X_z = np.clip(X_z, -3, 3) / 3.0

X_all = torch.tensor(X_z, dtype=torch.float32).unsqueeze(1)
y_all = torch.tensor(y_np, dtype=torch.long)

num_classes = len(np.unique(y_np))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ==== K-Fold Training ====
kfold = KFold(n_splits=5, shuffle=True, random_state=42)
all_val_true, all_val_pred = [], []

for fold_idx, (train_idx, val_idx) in enumerate(kfold.split(X_all)):
    print(f"\n===== Fold {fold_idx + 1} =====")

    # === Build and Prune Model ===
    model_unpruned = FinalNetwork(C=9, num_classes=num_classes, layers=7, genotype=searched_genotype).to(device)
    total_params_before = sum(p.numel() for p in model_unpruned.parameters())
    print(f"[INFO] Total parameters BEFORE pruning: {total_params_before:,}")

    model = prune_model_entropy(model_unpruned, prune_ratio=0.5)
    nonzero_params_after = sum((p != 0).sum().item() for p in model.parameters())
    zero_params = total_params_before - nonzero_params_after
    pruned_ratio = 100 * zero_params / total_params_before
    print(f"[INFO] Non-zero parameters AFTER pruning: {nonzero_params_after:,}")
    print(f"[INFO] Pruned parameters: {zero_params:,} ({pruned_ratio:.2f}%)")

    optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
    criterion = nn.CrossEntropyLoss()

    # Dataloader
    X_train, y_train = X_all[train_idx], y_all[train_idx]
    X_val, y_val = X_all[val_idx], y_all[val_idx]
    train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=64, shuffle=True)
    val_loader = DataLoader(TensorDataset(X_val, y_val), batch_size=64, shuffle=False)

    best_val_acc = 0.0
    best_result = {}
    no_improve_counter = 0  # ← EARLY STOPPING COUNTER

    for epoch in range(50):  # Max epochs
        model.train()
        train_loss = 0
        train_true, train_pred = [], []

        for x, y in train_loader:
            x, y = x.to(device).squeeze(-1), y.to(device)
            optimizer.zero_grad()
            output = model(x)
            loss = criterion(output, y)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            train_true.extend(y.cpu().numpy())
            train_pred.extend(output.argmax(dim=1).cpu().numpy())

        model.eval()
        val_loss = 0
        val_true, val_pred = [], []
        with torch.no_grad():
            for x, y in val_loader:
                x, y = x.to(device).squeeze(-1), y.to(device)
                output = model(x)
                loss = criterion(output, y)
                val_loss += loss.item()
                val_true.extend(y.cpu().numpy())
                val_pred.extend(output.argmax(dim=1).cpu().numpy())

        train_acc = accuracy_score(train_true, train_pred)
        val_acc = accuracy_score(val_true, val_pred)

        print(f"[Fold {fold_idx+1} | Epoch {epoch+1}] Train Loss: {train_loss/len(train_loader):.4f} | "
              f"Train Acc: {train_acc:.4f} | Val Loss: {val_loss/len(val_loader):.4f} | Val Acc: {val_acc:.4f}")

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            no_improve_counter = 0  # reset counter
            acc, mf1, mgm, prec, rec, f1s, gmeans, cm = evaluate_metrics(
                np.array(val_true), np.array(val_pred), num_classes
            )
            best_result = {
                'epoch': epoch + 1,
                'acc': acc,
                'mf1': mf1,
                'gmean': mgm,
                'prec': prec,
                'rec': rec,
                'f1': f1s,
                'gmean_class': gmeans,
                'cm': cm,
                'val_true': val_true,
                'val_pred': val_pred
            }
        else:
            no_improve_counter += 1
            if no_improve_counter >= 10:
                print(f"[Early Stopping] No improvement in 10 consecutive epochs. Stopping at epoch {epoch+1}.")
                break

    # === Print Best Result for Fold ===
    print(f"\n===== BEST RESULT FOR FOLD {fold_idx+1} =====")
    print(f"Best Epoch: {best_result['epoch']}")
    print(f"ACC: {best_result['acc']:.4f} | MF1: {best_result['mf1']:.4f} | G-Mean: {best_result['gmean']:.4f}")
    for i in range(num_classes):
        print(f"[Class {i}] Prec: {best_result['prec'][i]:.4f} | Rec: {best_result['rec'][i]:.4f} "
              f"| F1: {best_result['f1'][i]:.4f} | GM: {best_result['gmean_class'][i]:.4f}")

    # Gộp toàn bộ val_true và val_pred để đánh giá toàn bộ tập sau K-Fold
    all_val_true.extend(best_result['val_true'])
    all_val_pred.extend(best_result['val_pred'])

# ==== FINAL EVALUATION ON MERGED VAL SET ====
acc, mf1, mgm, prec, rec, f1s, gmeans, cm = evaluate_metrics(np.array(all_val_true), np.array(all_val_pred), num_classes)

print("\n===== FINAL EVALUATION ON MERGED TEST SET (AFTER K-FOLD) =====")
print(f"ACC: {acc:.4f} | MF1: {mf1:.4f} | G-Mean: {mgm:.4f}")
for i in range(num_classes):
    print(f"[Class {i}] Prec: {prec[i]:.4f} | Rec: {rec[i]:.4f} | F1: {f1s[i]:.4f} | GM: {gmeans[i]:.4f}")
print("Confusion Matrix:")
print(cm)



===== Fold 1 =====
[INFO] Total parameters BEFORE pruning: 75,902
[INFO] Non-zero parameters AFTER pruning: 38,254
[INFO] Pruned parameters: 37,648 (49.60%)
[Fold 1 | Epoch 1] Train Loss: 0.5965 | Train Acc: 0.7646 | Val Loss: 0.7113 | Val Acc: 0.7129
[Fold 1 | Epoch 2] Train Loss: 0.5316 | Train Acc: 0.7918 | Val Loss: 0.5037 | Val Acc: 0.8022
[Fold 1 | Epoch 3] Train Loss: 0.5160 | Train Acc: 0.7985 | Val Loss: 0.4847 | Val Acc: 0.8124
[Fold 1 | Epoch 4] Train Loss: 0.5058 | Train Acc: 0.8029 | Val Loss: 0.5024 | Val Acc: 0.8027
[Fold 1 | Epoch 5] Train Loss: 0.4965 | Train Acc: 0.8064 | Val Loss: 0.4893 | Val Acc: 0.8076
[Fold 1 | Epoch 6] Train Loss: 0.4872 | Train Acc: 0.8105 | Val Loss: 0.4818 | Val Acc: 0.8131
[Fold 1 | Epoch 7] Train Loss: 0.4813 | Train Acc: 0.8127 | Val Loss: 0.4598 | Val Acc: 0.8233
[Fold 1 | Epoch 8] Train Loss: 0.4748 | Train Acc: 0.8150 | Val Loss: 0.4613 | Val Acc: 0.8207
[Fold 1 | Epoch 9] Train Loss: 0.4702 | Train Acc: 0.8171 | Val Loss: 0.4640 | Val

[Fold 3 | Epoch 9] Train Loss: 0.4766 | Train Acc: 0.8141 | Val Loss: 0.4862 | Val Acc: 0.8099
[Fold 3 | Epoch 10] Train Loss: 0.4738 | Train Acc: 0.8159 | Val Loss: 0.4735 | Val Acc: 0.8188
[Fold 3 | Epoch 11] Train Loss: 0.4687 | Train Acc: 0.8174 | Val Loss: 0.4732 | Val Acc: 0.8116
[Fold 3 | Epoch 12] Train Loss: 0.4649 | Train Acc: 0.8205 | Val Loss: 0.4821 | Val Acc: 0.8116
[Fold 3 | Epoch 13] Train Loss: 0.4623 | Train Acc: 0.8209 | Val Loss: 0.4651 | Val Acc: 0.8234
[Fold 3 | Epoch 14] Train Loss: 0.4592 | Train Acc: 0.8225 | Val Loss: 0.4528 | Val Acc: 0.8228
[Fold 3 | Epoch 15] Train Loss: 0.4567 | Train Acc: 0.8228 | Val Loss: 0.4620 | Val Acc: 0.8185
[Fold 3 | Epoch 16] Train Loss: 0.4530 | Train Acc: 0.8250 | Val Loss: 0.4597 | Val Acc: 0.8250
[Fold 3 | Epoch 17] Train Loss: 0.4512 | Train Acc: 0.8248 | Val Loss: 0.4438 | Val Acc: 0.8278
[Fold 3 | Epoch 18] Train Loss: 0.4463 | Train Acc: 0.8276 | Val Loss: 0.4536 | Val Acc: 0.8243
[Fold 3 | Epoch 19] Train Loss: 0.4444 | 

[Fold 5 | Epoch 21] Train Loss: 0.4363 | Train Acc: 0.8310 | Val Loss: 0.4827 | Val Acc: 0.8126
[Fold 5 | Epoch 22] Train Loss: 0.4351 | Train Acc: 0.8322 | Val Loss: 0.4371 | Val Acc: 0.8310
[Fold 5 | Epoch 23] Train Loss: 0.4320 | Train Acc: 0.8334 | Val Loss: 0.4400 | Val Acc: 0.8295
[Fold 5 | Epoch 24] Train Loss: 0.4307 | Train Acc: 0.8319 | Val Loss: 0.4552 | Val Acc: 0.8239
[Fold 5 | Epoch 25] Train Loss: 0.4288 | Train Acc: 0.8338 | Val Loss: 0.4489 | Val Acc: 0.8290
[Fold 5 | Epoch 26] Train Loss: 0.4253 | Train Acc: 0.8350 | Val Loss: 0.4450 | Val Acc: 0.8291
[Fold 5 | Epoch 27] Train Loss: 0.4224 | Train Acc: 0.8357 | Val Loss: 0.4559 | Val Acc: 0.8223
[Fold 5 | Epoch 28] Train Loss: 0.4217 | Train Acc: 0.8365 | Val Loss: 0.4492 | Val Acc: 0.8257
[Fold 5 | Epoch 29] Train Loss: 0.4203 | Train Acc: 0.8368 | Val Loss: 0.4512 | Val Acc: 0.8232
[Fold 5 | Epoch 30] Train Loss: 0.4168 | Train Acc: 0.8386 | Val Loss: 0.4614 | Val Acc: 0.8266
[Fold 5 | Epoch 31] Train Loss: 0.4151 |